**SAVE AND LOAD MODELS** 

In [2]:
# install dan import tf dan dependensi
!pip install pyyaml h5py

In [4]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.7.0


In [7]:
# import dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11501568/11490434 [==============================] - 0s 0us/step


In [9]:
# membuat model
def create_model():
  model = tf.keras.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
  ])
  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])
  return model

# membuat basic model instance
model = create_model()

# menampilkan arsitektur model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Menyimpan Checkpoints selama Pelatihan

# penggunaan checkpoint callback
checkpoint_path = 'training_1/cp/ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# membuat callback yang menyimpan model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# melatih model dengan callback yang baru
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]) # pass callback ke training

# proses ini mungkin memunculkan warning yg berkaitan dengan penyimpanan state dari optimizer
# warnings ini dapat dihiraukan

Epoch 1/10
25/32 [======================>.......] - ETA: 0s - loss: 1.3258 - sparse_categorical_accuracy: 0.6112
Epoch 00001: saving model to training_1/cp/ckpt
32/32 [==============================] - 1s 20ms/step - loss: 1.1975 - sparse_categorical_accuracy: 0.6440 - val_loss: 0.7226 - val_sparse_categorical_accuracy: 0.7900
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4367 - sparse_categorical_accuracy: 0.8720
Epoch 00002: saving model to training_1/cp/ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.4372 - sparse_categorical_accuracy: 0.8710 - val_loss: 0.5486 - val_sparse_categorical_accuracy: 0.8240
Epoch 3/10
24/32 [=====================>........] - ETA: 0s - loss: 0.2942 - sparse_categorical_accuracy: 0.9232
Epoch 00003: saving model to training_1/cp/ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.2899 - sparse_categorical_accuracy: 0.9210 - val_loss: 0.4742 - val_sparse_categorical_accuracy: 0.8490
Epoch 4/10
26/3

In [11]:
os.listdir(checkpoint_dir)

['ckpt.index', 'ckpt.data-00000-of-00001', 'checkpoint']

In [12]:
# Sekarang, bangun kembali model yang fresh dan untrained dan evaluate di test set

# membuat basic model instance
model = create_model()

#evaluasi model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 2.3082 - sparse_categorical_accuracy: 0.1360 - 153ms/epoch - 5ms/step
Untrained model, accuracy: 13.60%


In [14]:
# Load the weights
model.load_weights(checkpoint_path)

# re-evaluasi model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4165 - sparse_categorical_accuracy: 0.8680 - 66ms/epoch - 2ms/step
Restored model, accuracy: 86.80%


In [15]:
# Opsi callback checkpoint

# masukkan epoch ke dalam file name (gunakan 'str.format')
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# membuat callback yang menyimpan model's weights setiap 5 epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)
# membuat model instance baru
model = create_model()

# menyimpan weights menggunakan format 'checkpoint_path'
model.save_weights(checkpoint_path.format(epoch=0))

# melatih model dengan callback yang baru
model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [16]:
os.listdir(checkpoint_dir)

['cp-0015.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0040.ckpt.index']

In [17]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [18]:
# Reset model dan load latest checkpoint
model = create_model()

# muat weights yang telah disimpan sebelumnya
model.load_weights(latest)

# reevaluate model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, akurasi: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4642 - sparse_categorical_accuracy: 0.8810 - 152ms/epoch - 5ms/step
Restored model, akurasi: 88.10%


Kode diatas menyimpan the weights ke dalam koleksi file berformat-checkpoint yang mengandung hanya trained weights dalam format binary. Checkpoint mengandung:


*   Satu atau lebih shards yang mengandung model weights 
*   Sebuah indeks yang mengindikasikan bobot mana yang disimpan dalam salah satu shard



In [19]:
# Menyimpan BObot Manual

# menyimpan bobot
model.save_weights('checkpoints/my_checkpoint')

# membuat model instan baru
model = create_model()

# mengembalikan bobot
model.load_weights('./checkpoints/my_checkpoint')

# evaluasi model
loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, akurasi: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4642 - sparse_categorical_accuracy: 0.8810 - 155ms/epoch - 5ms/step
Restored model, akurasi: 88.10%


In [20]:
# Menyimpan Seluruh MOdel

# SavedModel format

# membuat dan melatih model instan
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# save seluruh model sebagai SavedModel
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1496 - sparse_categorical_accuracy: 0.6730
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4283 - sparse_categorical_accuracy: 0.8750
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2903 - sparse_categorical_accuracy: 0.9190
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2113 - sparse_categorical_accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1476 - sparse_categorical_accuracy: 0.9690
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [23]:
# my_model directory
%ls saved_model

# mengandung folder assets, saved_model.pb dan variables
%ls saved_model/my_model

my_model/
assets/  keras_metadata.pb  saved_model.pb  variables/


In [24]:
# reload model Keras baru dari model tersimpan
new_model = tf.keras.models.load_model('saved_model/my_model')

# mengecek arsitekturnya
new_model.summary() 

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Mengevaluasi restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, akurasi: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4292 - sparse_categorical_accuracy: 0.8570 - 160ms/epoch - 5ms/step
Restored model, akurasi: 85.70%
(1000, 10)


In [26]:
# Save format menggunakan HDF5 standard

# membuat dan melatih model instan
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# menyimpan seluruh model dalam file HDF5 (memperlihatkan ekstensi '.h5')
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1434 - sparse_categorical_accuracy: 0.6710
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4241 - sparse_categorical_accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2881 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2169 - sparse_categorical_accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1617 - sparse_categorical_accuracy: 0.9650


In [27]:
# membuat file dari file diatas

# membuat ulang model yg sama, termasuk bobot dan optimizer nya
new_model = tf.keras.models.load_model('my_model.h5')

# tampilkan arsitektur model
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [28]:
# mengecek akurasi
loss,acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, akurasi: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4288 - sparse_categorical_accuracy: 0.8620 - 153ms/epoch - 5ms/step
Restored model, akurasi: 86.20%


Keras saves models by inspecting their architectures. This technique saves everything:

1. The weight values
2. The model's architecture
3. The model's training configuration (what you pass to the .compile() method)
4. The optimizer and its state, if any (this enables you to restart training where you left off)


